In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate
from sklearn.preprocessing import LabelEncoder
import numpy as np

2023-08-14 08:35:14.874692: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-14 08:35:17.342757: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-14 08:35:17.352830: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-14 08:35:21.183149: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
filename='230814_intent_labeled_by_7'

train_file = f'/home/azureuser/projects/aerius/ai/myapp/chatbot/datas/{filename}.csv'
data = pd.read_csv(train_file, delimiter = ',')
queries = data['query'].tolist()
intents = data['label'].tolist()

print(data.head(5))
print('len of queries = ', len(queries))
print('len of intents = ', len(intents))

                              query intent  label
0  아침도 아니고 밤 12시 30분에 결제한 건데도 그런가요?     주문      0
1             실수로 취소하면 재주문해야 하는 거죠?     주문      0
2                     택배비 따로 추가되나요?     배송      1
3                          택배비 있나요?     배송      1
4                        택배비 따로 들어요     배송      1
len of queries =  176625
len of intents =  176625


In [3]:
train_data = data.sample(frac=0.7, random_state=42)
temp_data = data.drop(train_data.index)
val_data = temp_data.sample(frac=0.66, random_state=42)
test_data = temp_data.drop(val_data.index)

# Oversampling the minority classes in training data
max_size = train_data['label'].value_counts().max()
lst = [train_data]
for class_index, group in train_data.groupby('label'):
    lst.append(group.sample(max_size-len(group), replace=True))
train_data_oversampled = pd.concat(lst)

# Checking the distribution after oversampling
oversampled_distribution = train_data_oversampled['label'].value_counts()

oversampled_distribution

label
1    43535
4    43535
6    43535
5    43535
3    43535
2    43535
0    43535
Name: count, dtype: int64

In [4]:
queries = train_data_oversampled['query'].tolist()
intents = train_data_oversampled['label'].tolist()

print(train_data_oversampled.head(5))
print('len of queries = ', len(queries))
print('len of intents = ', len(intents))

                                             query intent  label
32942                     13일 주문했는데 언제쯤 받을 수 있을까요?     배송      1
126485  방금 추천해 주신 헤어오일 향이 너무 좋은데, 무슨 향인지 알려 주시겠어요?  제품_정보      4
154437       배송 준비 중인데 내일 도착 예정이라고 뜨네요 낼 받을 수 있나요?     배송      1
65307                           105싸이즈가입고가 되긴 할까요?  제품_기타      6
75189                            겨울왕국 그려진 아동복 있어요?  제품_재고      5
len of queries =  304745
len of intents =  304745


In [5]:
encoder = LabelEncoder()
encoder.fit(intents)

encoded_intents = encoder.transform(intents)
num_classes = len(encoder.classes_)
print(f"The number of unique labels is {num_classes}")

The number of unique labels is 7


In [6]:
from ai.myapp.chatbot.utils.Preprocess import Preprocess
p = Preprocess()

words = []
for sentence in queries:
    if isinstance(sentence, str):  # Only process if the sentence is a string
        preprocessed = p.delete_intent_trash_tags(sentence=sentence)
        word_list, _ = p.divide_words_tags(preprocessed)
        words.extend(word_list)
    else:
        print(f"Found non-string value: {sentence}")


# Initialize the tokenizer
p.initialize_tokenizer(words)

# Convert the queries into sequences
sequences = []
for sentence in queries:
    sequence = p.text_to_sequence(sentence)
    sequences.append(sequence)

None


In [7]:
from ai.backend.settings import INTENT_MAX_SEQ_LEN
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=INTENT_MAX_SEQ_LEN, padding='post')

ds = tf.data.Dataset.from_tensor_slices((padded_seqs, encoded_intents))
ds = ds.shuffle(len(queries))

train_size = int(len(padded_seqs)*0.7)
val_size = int(len(padded_seqs)*0.2)
test_size = int(len(padded_seqs)*0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(p.tokenizer.word_index) + 1

input_layer = Input(shape=(INTENT_MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=INTENT_MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(filters = 64,
               kernel_size = 3,
               padding = 'valid',
               activation = tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(filters = 64,
               kernel_size = 4,
               padding = 'valid',
               activation = tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)


concat = concatenate([pool1, pool2])

hidden = Dense(64, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(num_classes, name='logits')(dropout_hidden)
predictions = Dense(num_classes, activation=tf.nn.softmax)(logits)

model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds, validation_data = val_ds, epochs = EPOCH, verbose=1)

Epoch 1/5
10667/10667 [==============================] - 265s 24ms/step - loss: 1.1137 - accuracy: 0.5723 - val_loss: 0.8946 - val_accuracy: 0.6559
Epoch 2/5
10667/10667 [==============================] - 309s 29ms/step - loss: 0.9581 - accuracy: 0.6350 - val_loss: 0.8382 - val_accuracy: 0.6761
Epoch 3/5
10667/10667 [==============================] - 308s 29ms/step - loss: 0.9123 - accuracy: 0.6522 - val_loss: 0.8082 - val_accuracy: 0.6875
Epoch 4/5
10667/10667 [==============================] - 322s 30ms/step - loss: 0.8865 - accuracy: 0.6618 - val_loss: 0.7772 - val_accuracy: 0.6991
Epoch 5/5
10667/10667 [==============================] - 359s 34ms/step - loss: 0.8630 - accuracy: 0.6703 - val_loss: 0.7586 - val_accuracy: 0.7026


In [8]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# 1. 모델 예측
y_pred = model.predict(test_ds)
y_pred_classes = np.argmax(y_pred, axis=1)

# test_ds_resampled에서 라벨만 추출
y_true = np.concatenate([y for x, y in test_ds], axis=0)

# 2. 성능 지표 계산
precision = precision_score(y_true, y_pred_classes, average='weighted', zero_division=1)
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')
accuracy = accuracy_score(y_true, y_pred_classes)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

1524/1524 [==============================] - 30s 16ms/step
Precision: 0.1448
Recall: 0.1448
F1 Score: 0.1420
Accuracy: 0.1448


In [9]:
loss, accuracy = model.evaluate(test_ds, verbose=0)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 0.7649316787719727
Test accuracy: 0.700564444065094


In [10]:
loss, accuracy = model.evaluate(test_ds, verbose=1)
print(f'Accuracy: {accuracy * 100}')
print(f'loss: {loss}')

model.save(f'/home/azureuser/projects/aerius/ai/myapp/chatbot/models/{filename}_lower_dense.h5')

1524/1524 [==============================] - 22s 14ms/step - loss: 0.7583 - accuracy: 0.7047
Accuracy: 70.47318816184998
loss: 0.7583162188529968


/home/azureuser/projects/aerius/venv/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
